In [2]:
#!pip3 install "psycopg[binary]"

In [19]:
import config
import psycopg
import pytz
import pandas as pd
import numpy as np
from psycopg.rows import dict_row

In [4]:
def pgDictToConn(secretDict):
    pgStrs = []
    for key in secretDict:
        pgStrs.append('{}={}'.format(key, secretDict[key]))
    return ' '.join(pgStrs)


In [5]:
#Init
pgConnStr = pgDictToConn(config.pgSecrets)

In [31]:
def getTicker(symbol):
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT * FROM alpaca_minute WHERE symbol = %s'''
            data = (symbol, )
            result = cur.execute(stmt, data).fetchall()
            desc = cur.description
            cols = [col[0] for col in desc]
            conn.commit()
    df = pd.DataFrame(data=result, columns=cols)
#     df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')
    df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')
    return df #pd.DataFrame(data=result, columns=cols)

In [29]:
def getTickerList():
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT DISTINCT symbol FROM alpaca_minute ORDER BY symbol'''
            result = cur.execute(stmt).fetchall()
            conn.commit()
            return [row['symbol'] for row in result]

In [84]:
#Get List of all tickers
len(getTickerList())

18

In [85]:
#Sample 10 random stocks
tickers = getTickerList()
random_pick = list(np.random.choice(tickers, 10 if len(tickers) > 10 else  len(tickers), replace=False))
random_pick

['GNRC',
 'GNFT',
 'GNLN',
 'SF',
 'GNSS',
 'GNE',
 'GMRE',
 'GNE.PRA',
 'GMVD',
 'AAPL']

In [86]:
getTicker(random_pick[0])

,id,date,symbol,open,close,high,low,trade_count,vol,vwap,datetime
0,242070,2022-09-15,GNRC,217.360,217.3600,217.390,217.360,75.0,2466.0,217.372458,2022-09-15 10:26:00-04:00
1,242071,2022-09-15,GNRC,217.385,214.7100,217.385,214.710,313.0,11165.0,216.347007,2022-09-15 10:27:00-04:00
2,242072,2022-09-15,GNRC,214.690,214.6089,214.690,214.460,41.0,1290.0,214.573392,2022-09-15 10:28:00-04:00
3,242073,2022-09-15,GNRC,214.643,214.4600,214.643,214.340,81.0,3225.0,214.475613,2022-09-15 10:29:00-04:00
4,242074,2022-09-15,GNRC,214.110,214.6400,214.750,214.090,111.0,3034.0,214.416162,2022-09-15 10:30:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...
44833,286903,2023-03-13,GNRC,113.430,113.5200,113.530,113.430,227.0,14678.0,113.474096,2023-03-13 15:58:00-04:00
44834,286904,2023-03-13,GNRC,113.520,113.6500,113.720,113.515,546.0,32507.0,113.604752,2023-03-13 15:59:00-04:00
44835,286905,2023-03-13,GNRC,113.640,113.6400,113.640,113.640,25.0,88443.0,113.639992,2023-03-13 16:00:00-04:00
44836,286906,2023-03-13,GNRC,113.640,113.6400,113.640,113.640,12.0,1056.0,113.640000,2023-03-13 16:01:00-04:00


In [53]:
#Get Specific Ticker
AAPL = getTicker('AAPL')

In [71]:
AAPL.loc[390,:]

id                                   393
date                          2022-09-15
symbol                              AAPL
open                              151.89
close                             151.73
high                              151.89
low                               151.73
trade_count                        163.0
vol                              20313.0
vwap                          151.822464
datetime       2022-09-15 16:56:00-04:00
Name: 390, dtype: object